In [35]:
import pdfminer
from subprocess import call
import fileinput
import uuid
import itertools
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
pdfname = 'standings.pdf'
standings_out = 'standings.txt'
standings_cleaned = 'cleanedstandings.txt'
point_types = ['major', 'open', 'ladder', 'playoff', 'championship_seasonal', 'championship_world']

In [12]:
pdfname = 'lastcall.pdf'
standings_out = 'lastcall.txt'
standings_cleaned = 'cleanedlastcall.txt'
point_types = ['major', 'open', 'ladder', 'playoff', 'championship_seasonal', 'championship_world']

In [13]:
def is_int(s):
    try:
        int(s)
    except ValueError:
        return False
    return True

In [18]:
#May have to modify pdf2txt.py to use unix line endings instead of windows endings
call(['pdf2txt.py', '-o', standings_out,pdfname])

0

In [38]:
#DONT RUN THIS IF YOU CHANGED CLEANED STANDINGS MANUALLY
#tr '\f' '\n' < standings.txt | sed '/^$/d' > cleanedstandings.txt
#for line in fileinput.FileInput("standings1.txt", inplace=True):
text = ''
for line in open(standings_out, 'r'):
    if line.rstrip() and not line.startswith('This p') and \
        not line.startswith('comments, o') and not line.startswith('questions, c'):
        if line[0]=='\f':
            line = line[1:]
        arr = line.split()
        if arr[0]=='-' and is_int(arr[1]):
            text+=' '.join(arr[1:])+'\n'
        elif arr[1]=='Qualified':
            continue
        elif arr[1]!='has':
            text = text[:-1] #Remove the newline and join with next line
            text+=line
        else:
            text+=line
with open(standings_cleaned, 'w') as out:
    out.write(text)

In [21]:
regions = {}
regions['AM'] = 'Americas'
regions['AMERICAS'] = 'Americas'
regions['EU'] = 'Europe'
regions['EUROPE'] = 'Europe'
regions['ASIA_PACIFIC'] = 'Asia-Pacific'
regions['AP'] = 'Asia-Pacific'
get_region = lambda x : regions[x.upper()]

In [26]:
user = None
users = {}
for line in open(standings_cleaned, 'r'):
    if is_int(line.split()[0]):
        if user is None:
            raise ValueError('Invalid state')
        else:
            arr = line.split()
            points = int(arr[0])
            #Open win
            if arr[4]=='placing':
                tournament = ' '.join(arr[8:-2])
                month = arr[-1]
                
                if arr[-3] == 'Playoffs':
                    if arr[5]=='5th/8th' or arr[5]=='1st/4th':
                        place = arr[5]
                    else:
                        place = ' '.join(arr[5:8])
                        tournament = ' '.join(arr[10:-2])
                    point_type = 'playoff'
                elif 'Championship' in tournament:
                    place = arr[5]
                    point_type = 'championship_seasonal'
                elif 'Dreamhack' in tournament or 'Major' in tournament or 'Copa America' in tournament:
                    point_type = 'major'
                    place = arr[5]
                else:
                    point_type = 'open'
                    place = arr[5]
                
                point = {'type':point_type, 'points':points, 'place':place, 'tournament':tournament,
                        'month':month}
            elif arr[4]=='reaching':
                point_type = 'ladder'
                place = arr[5]
                month = arr[-2]
                server = get_region(arr[-1][1:-1])
                point = {'type':point_type, 'points':points, 'rank':place,
                        'month':month, 'server':server}
            else:
                raise ValueError('Invalid type')
            user['breakdown'].append(point)
    else:
        if user is not None:
            users[userId] = user
        arr = line.split()
        userId = uuid.uuid4()
        username = arr[0]
        points = int(arr[4])
        if arr[9]=='Last':
            season = arr[9:11]
            region = arr[12][1:-1]
        else:
            season = arr[9]
            region = arr[11][1:-1]
        region = get_region(region)
        breakdown = []
        #user = {'name':username, 'userId':userId, 'points':points, 'season':season, 'region':region, 
        #        'breakdown': breakdown}
        user = {'name':username, 'points':points, 'season':season, 'region':region, 
                'breakdown': breakdown}

users[userId] = user

In [27]:
for user in users.values():
    if user['points']!=sum([x['points'] for x in user['breakdown']]):
        print(user['name'])

In [28]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    num_users = 100
    filtered = filtered[:num_users]
    print('Leaderboard for {}'.format(region))
    
    count = 0
    for i, user in enumerate(filtered):
        print('{:<4}{:<15}{}'.format(i+1, user['name'], user['points']))

Leaderboard for Americas
1   muzzy          106
2   Zalae          105
3   amnesiasc      88
4   Astrogation    86
5   Apxvoid        80
6   PNC            77
7   Rase           76
8   Monsanto       70
9   killinallday   68
10  rayC           67
11  Cydonia        66
12  DrJikininki    64
13  seohyun628     60
14  Tarei          60
15  Fr0zen         59
16  Nalguidan      59
17  Qwerty97       57
18  baenugget      57
19  Kuonet         56
20  Rosty          55
21  yoitsflo       55
22  noblord        55
23  Teebs          53
24  pksnow         53
25  Fibonacci      52
26  WoT            52
27  Stark          52
28  Fenom          51
29  Th3RaT         51
30  justsaiyan     51
31  shoop          50
32  topopablo11    50
33  Rpbalance      49
34  Pinche         48
35  Ant            46
36  Villain        46
37  Echn2          46
38  Cheese         46
39  lnguagehackr   45
40  leadpaint      45
41  CelticGuard    45
42  Vlps           45
43  Korextron      45
44  Alan870806     44
45  D

In [41]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    num_users = 64
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    point_cutoff = filtered[num_users-1]['points']
    filtered = list(filter(lambda x:x['points']>=point_cutoff, filtered))
    with open('top64{}.json'.format(region),'w', encoding='utf-8') as f:
        f.write(json.dumps(filtered, ensure_ascii=False))

In [40]:
counts = {}
for user in users.values():
    for point in user['breakdown']:
        if point['type']=='ladder':
            key = (point['server'], point['month'])
            if key not in counts:
                counts[key] = 0
            counts[key]+=1
regions = ['Americas', 'Europe', 'Asia-Pacific']
months = ['Nov','Dec', 'Jan','Feb','Mar','Apr','May','June', 'July']
for x in itertools.product(regions, months):
    print('Server {} had {} legend players in {}'.format(x[0],counts[x],x[1]))

Server Americas had 2989 legend players in Nov
Server Americas had 6059 legend players in Dec
Server Americas had 3241 legend players in Jan
Server Americas had 1729 legend players in Feb
Server Americas had 5251 legend players in Mar
Server Americas had 12107 legend players in Apr
Server Americas had 6166 legend players in May
Server Americas had 3479 legend players in June
Server Americas had 3218 legend players in July
Server Europe had 8599 legend players in Nov
Server Europe had 12679 legend players in Dec
Server Europe had 8364 legend players in Jan
Server Europe had 4592 legend players in Feb
Server Europe had 13235 legend players in Mar
Server Europe had 27250 legend players in Apr
Server Europe had 15568 legend players in May
Server Europe had 8306 legend players in June
Server Europe had 7740 legend players in July
Server Asia-Pacific had 3413 legend players in Nov
Server Asia-Pacific had 6068 legend players in Dec
Server Asia-Pacific had 3424 legend players in Jan
Server Asi

In [43]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    num_users = 100
    point_cutoff = filtered[num_users-1]['points']
    filtered = list(filter(lambda x:x['points']>=point_cutoff, filtered))
    print('Of the top {} players from {}'.format(len(filtered), region))
    count = 0
    for user in filtered:
        count+=len(list(filter(lambda x:x['type']=='open' and x['points']==0, user['breakdown'])))
    print('An average of {} opens did not count'.format(count/len(filtered)))

Of the top 104 players from Americas
An average of 3.1346153846153846 opens did not count
Of the top 100 players from Europe
An average of 3.41 opens did not count
Of the top 104 players from Asia-Pacific
An average of 1.6346153846153846 opens did not count


In [44]:
cross = {}
for user in users.values():
    for point in user['breakdown']:
        if point['type']=='ladder' and point['points']>=4:
            key = (user['region'], point['server'])
            if key not in cross:
                cross[key]=0
            cross[key]+=1
for a,b in itertools.product(['Americas', 'Europe', 'Asia-Pacific'],['Americas', 'Europe', 'Asia-Pacific']):
    if (a,b) in cross:
        print('{} players from {} finished on {}'.format(cross[(a,b)], a, b))

155 players from Americas finished on Americas
4 players from Americas finished on Europe
7 players from Americas finished on Asia-Pacific
60 players from Europe finished on Americas
276 players from Europe finished on Europe
17 players from Europe finished on Asia-Pacific
78 players from Asia-Pacific finished on Americas
3 players from Asia-Pacific finished on Europe
259 players from Asia-Pacific finished on Asia-Pacific


In [13]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    num_users = 64
    point_cutoff = filtered[num_users-1]['points']
    filtered = list(filter(lambda x:x['points']>=point_cutoff, filtered))
    print('The top {} players from {}'.format(len(filtered), region))
    
    count = 0
    for user in filtered:
        count+=sum([x['points'] for x in user['breakdown'] if x['type']=='open'])
    print('earned an average of {} points from opens'.format(count/len(filtered)))

The top 65 players from Americas
earned an average of 8.907692307692308 points from opens
The top 70 players from Europe
earned an average of 10.371428571428572 points from opens
The top 67 players from Asia-Pacific
earned an average of 6.955223880597015 points from opens


In [25]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    num_users = 64
    point_cutoff = filtered[num_users-1]['points']
    filtered = list(filter(lambda x:x['points']>=point_cutoff, filtered))
    print('The top {} players from {}'.format(len(filtered), region))
    
    count = 0
    for user in filtered:
        count+=sum([x['points'] for x in user['breakdown'] if x['type']=='ladder'])
    print('earned an average of {} points from ladder'.format(count/len(filtered)))

The top 65 players from Americas
earned an average of 10.307692307692308 points from ladder
The top 70 players from Europe
earned an average of 14.342857142857143 points from ladder
The top 67 players from Asia-Pacific
earned an average of 13.194029850746269 points from ladder


In [24]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    num_users = 64
    point_cutoff = filtered[num_users-1]['points']
    filtered = list(filter(lambda x:x['points']>=point_cutoff, filtered))
    print('The top {} players from {}'.format(len(filtered), region))
    
    count = 0
    for user in filtered:
        count+=sum([x['points'] for x in user['breakdown'] if x['type']=='major'])
    print('earned an average of {} points from majors'.format(count/len(filtered)))

The top 65 players from Americas
earned an average of 1.2769230769230768 points from majors
The top 70 players from Europe
earned an average of 1.0142857142857142 points from majors
The top 67 players from Asia-Pacific
earned an average of 0.8805970149253731 points from majors


In [46]:
regionFilter = lambda region: lambda user: user['region']==region
for region in ['Americas', 'Europe', 'Asia-Pacific']:
    filtered = list(filter(regionFilter(region), users.values()))
    filtered.sort(key=lambda user:-user['points'])
    num_users = 64
    point_cutoff = filtered[num_users-1]['points']
    filtered = list(filter(lambda x:x['points']>=point_cutoff, filtered))
    print('Of the top {} players from {}'.format(len(filtered), region))
    
    count = 0
    for user in filtered:
        count+= sum([1 if x['type']=='playoff' else 0 for x in user['breakdown']])
    print('{} people requalified for prelims from spring'.format(count))

Of the top 65 players from Americas
25 people requalified for prelims from spring
Of the top 70 players from Europe
24 people requalified for prelims from spring
Of the top 67 players from Asia-Pacific
36 people requalified for prelims from spring
